# Problem 1

## Using pandas to answer a question from the US government College Scorecard

It is common for colleges in the United States to charge different tuition and fees for those students who are state residents versus "out-of-state" students.

https://www.collegexpress.com/interests/public-colleges-and-universities/blog/what-you-need-know-about-out-state-tuition/

The US government has assembled a database of college information called the "College Scorecard."

Using the College Scorecard data, you will answer the question:

> How do US colleges and universities which offer bachelor's degrees in engineering compare in their treatment of in-state and out-of-state tuition and fees?

## Final product

Your final product will be a pandas dataframe of college names sorted in descending order by the percent difference between in-state and out-of-state tuition

## Raw data sources

Your "raw" data sources are both from https://collegescorecard.ed.gov, with some modifications to optimize the data for this midterm:

1. a copy of the "Most-Recent-Cohorts-All-Data-Elements.csv" downloaded 2019-03-05T21:30Z
2. a slightly massaged version of the "CollegeScorecardDataDictionary.xlsx" that has been converted to a tab-delimited text file and reshaped for this midterm question

### Most-Recent-Cohorts-All-Data-Elements.csv

This ~140MB file has the data you need to produce your final product. 

Unfortunately, it has

1. a lot MORE than you need, 
2. column labels that are not human-readable, and
3. a lot of missing data in the form of either "NULL" entries or "PrivacySuppressed" entries

### CollegeScorecardDataDictionary_wide.txt

To help you decipher the contents of the data file, the "Data Dictionary" is provided. This is __NOT__ a python dictionary. Rather, it is a tab-delimited table that describes the column labels in the data file, as well as a "decoder ring" for the numeric codes used in some of the data columns.

Unfortunately, it has

1. a lot MORE information than you need,
2. "decoder ring" information is stored in a very inconvenient and inefficient "wide" format

In [1]:
import pandas as pd
import ast
import os
import csv

# Ignore this line. It will be used later.
SAVE_ASSIGN = getattr(pd.DataFrame, 'assign')

#### Exercise 1 (1 point)

Given the filename, write a function to import the "data dictionary" tab-delimited file into a pandas dataframe.

In [2]:
def tab_import(fname):
    df=pd.read_csv(fname, sep='\t')
    return df

In [3]:
# Test Cell: `tab_import_test`

from pandas.util import hash_pandas_object

dd_df = tab_import('../resource/lib/publicdata/mt2p1/CollegeScorecardDataDictionary_wide.txt')

assert hash_pandas_object(dd_df).sum() == 691811159278074823, "You have not imported the data correctly..."
display(dd_df.head())
print("\nPassed!")

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,-2,-1,0,1,2,...,98,99,100,101,102,103,104,105,106,107
0,Flag for Asian American Native American Pacifi...,school,minority_serving.aanipi,integer,AANAPII,NaN,NaN,No,Yes,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Accreditor for institution,school,accreditor,string,ACCREDAGENCY,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Code corresponding to accreditor (as captured ...,school,accreditor_code,string,ACCREDCODE,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25th percentile of the ACT cumulative score,admissions,act_scores.25th_percentile.cumulative,float,ACTCM25,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75th percentile of the ACT cumulative score,admissions,act_scores.75th_percentile.cumulative,float,ACTCM75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Passed!


The above cell displays the first 5 rows of the data dictionary. Each row of this dataframe contains details of each column label of 'Most-Recent-Cohorts-All-Data-Elements.csv'. The 'VARIABLE NAME' column contains the names currently used as column labels in the 'Most-Recent-Cohorts-All-Data-Elements.csv' file, while the 'developer-friendly name' column contains the labels for those 'variables' that are more understandable.

The numeric column labels are Codes and each column contains a row holding a description of what the code means. 

We will be using the 'Most-Recent-Cohorts-All-Data-Elements.csv' file later on in the problem. 

Now, as you may have noticed, this is a 1.7+MB dataframe that consists primarily of NaNs in a "wide" format.

The next few exercises will transform the dataframe into a cleaner and more useful format.

#### Exercise 2 (1 point)

Write a function that 

1. Adds a column to the df called 'Coded' that contains Boolean values: False if the columns labeled '-2' through '107' are all NaN, and True otherwise
2. Splits the df into two new dataframes, one containing the rows that are 'Coded' and those that are not
3. Removes the 'Coded' column from each new dataframe, since it is no longer required

In [4]:
def split_by_coded(full_df):
    
    full_df["Coded"]=~dd_df.loc[:, '-2':'107' ].isna().all(1)
    coded_df=full_df[full_df["Coded"]==True]
    not_coded_df=full_df[full_df["Coded"]==False]

    coded_df.drop("Coded",axis=1,inplace=True)
    not_coded_df.drop("Coded",axis=1,inplace=True)
    
    return coded_df, not_coded_df

In [5]:
# Test Cell: `split_by_coded_test`

from pandas.util import hash_pandas_object

c_df, n_c_df = split_by_coded(dd_df)

assert 'Coded' in dd_df.columns, "You have not added the column, 'Coded', to the data dictionary"
assert 'Coded' not in c_df and 'Coded' not in n_c_df, "You have not deleted the column, 'Coded', from the two new dataframes"

assert hash_pandas_object(c_df).sum() == -4867622688808165705 and hash_pandas_object(n_c_df).sum() == 5559433848086240528, "You have not split the data properly"
print("Passed!")

Passed!


/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


#### Exercise 3 (2 points)

Write a function that

1. Melts the coded dataframe columns '-2' to '107' into a column 'Code' containing the labels and a column 'Description' containing the code description
1. Removes all the rows in which 'Description' is NaN from the melted coded dataframe, since these rows add no information
1. Deletes the not_coded dataframe columns '-2' to '107' and adds two columns, labeled 'Code' and 'Description', to the not_coded dataframe, populating them with NaN's (use `np.nan`), since this dataframe had no codes to describe. It is recommended to use the `pandas.DataFrame.assign()` method. The `.assign()` method is useful for cleanly creating a new dataframe with new columns, optimized for "chaining" methods. https://stackoverflow.com/questions/48177914/why-use-pandas-assign-rather-than-simply-initialize-new-column
1. Concatenates the two new dataframes into a single dataframe
1. The columns should be ordered as __NAME OF DATA ELEMENT, dev-category, developer-friendly name, API data type, VARIABLE NAME,	Code, Description__.

Hint: Make sure to reset the index of the final melted dataframe!

In [12]:
import numpy as np

def melt(df, col_vals, key, value):
    assert type(df) is pd.DataFrame
    keep_vars=df.columns.difference(col_vals) # Get the names of the columns that we want to keep.  column names in the dataframe not in col_vals.
    melted_sections=[]                        # Empty dataframe known as melted_sections.
    for c in col_vals:                        # go through all the column values.
        melted_c=df[keep_vars].copy()         # first time in loop: 1999
        melted_c[key]=c                       # value column, wich is cases, set to original data frame indexed as original C
        melted_c[value]=df[c]                 
        melted_sections.append(melted_c)
    melted = pd.concat(melted_sections)
    return melted    


def melt_together(coded_df, not_coded_df):
    cdf=coded_df.copy()
    ncdf=not_coded_df.copy()
    
    cv=cdf.loc[0:0, '-2':'107' ].columns
    mcdf=melt(cdf,cv,"Code","Description")
    mask=mcdf.Description.notna()
    mcdf=mcdf[mask]
    
    ncdf.drop(cv,axis=1,inplace=True)
    ncdf["Code"]=np.nan
    ncdf["Description"]=np.nan
    
    cols=['NAME OF DATA ELEMENT','dev-category','developer-friendly name','API data type','VARIABLE NAME','Code', 'Description']

    frames=[mcdf,ncdf]
    melted_df=pd.concat(frames,sort=False)[cols]
    

    return melted_df


In [13]:
# Test Cell: `melt_together_test`

m_df = melt_together(c_df, n_c_df)

display(m_df.head())

assert 'Code' in m_df.columns, "The column, 'Code' is not present in the melted dataframe"
assert 'Description' in m_df.columns, "The column, 'Description' is not present in the melted dataframe"
#print( hash_pandas_object(m_df).sum())
assert hash_pandas_object(m_df).sum() == 5644814193110202630, "You have not carried out the melt operation correctly"
print("Passed!")

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,Code,Description
80,Carnegie Classification -- basic,school,carnegie_basic,integer,CCBASIC,-2,Not applicable
81,Carnegie Classification -- size and setting,school,carnegie_size_setting,integer,CCSIZSET,-2,Not applicable
82,Carnegie Classification -- undergraduate profile,school,carnegie_undergrad,integer,CCUGPROF,-2,Not applicable
1774,Religous affiliation of the institution,school,religious_affiliation,integer,RELAFFIL,-2,Not applicable
1774,Religous affiliation of the institution,school,religious_affiliation,integer,RELAFFIL,-1,Not reported


AssertionError: You have not carried out the melt operation correctly

In [14]:
# Test Cell: `melt_together_test`

m_df = melt_together(c_df, n_c_df)

display(m_df.head())

assert 'Code' in m_df.columns, "The column, 'Code' is not present in the melted dataframe"
assert 'Description' in m_df.columns, "The column, 'Description' is not present in the melted dataframe"
#print( hash_pandas_object(m_df).sum())
assert hash_pandas_object(m_df).sum() == 5644814193110202630, "You have not carried out the melt operation correctly"
print("Passed!")

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,Code,Description
80,Carnegie Classification -- basic,school,carnegie_basic,integer,CCBASIC,-2,Not applicable
81,Carnegie Classification -- size and setting,school,carnegie_size_setting,integer,CCSIZSET,-2,Not applicable
82,Carnegie Classification -- undergraduate profile,school,carnegie_undergrad,integer,CCUGPROF,-2,Not applicable
1774,Religous affiliation of the institution,school,religious_affiliation,integer,RELAFFIL,-2,Not applicable
1774,Religous affiliation of the institution,school,religious_affiliation,integer,RELAFFIL,-1,Not reported


AssertionError: You have not carried out the melt operation correctly

#### Exercise 4 (2 points)

Now that you have the "data dictionary" in a nicer form, use it to write a function that returns a 5-element tuple containing:

1. The string value for the '__VARIABLE NAME__' that corresponds to the '__developer-friendly name__', 'name'. This is the name of the column that contains college names.
2. The string value for the '__VARIABLE NAME__' that corresponds to the '__developer-friendly name__', 'tuition.in_state'. This is the name of the column that contains the in-state tuition amount for each college.
3. The string value for the '__VARIABLE NAME__' that corresponds to the '__developer-friendly name__', 'tuition.out_of_state'. This is the name of the column that contains the out-of-state tuition amount for each college.
4. The string value for the '__VARIABLE NAME__' that corresponds to the '__developer-friendly name__', 'program.bachelors.engineering'. This is the name of the column that contains all the colleges that offer a bachelor's degree in engineering.
5. The float value of the __Code__ for a college that offers a bachelor's degree in engineering. Hence, the 'Description' corresponding to this data point must be 'Program offered'.

For example, suppose the given developer-friendly name is 'carnegie_basic', the corresponding VARIABLE NAME is 'CCBASIC'.

__Note:__ This may require some initial exploration of the dataframe. You may find this useful http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.contains.html?highlight=contains#pandas.Series.str.contains

In [17]:
def get_dd_info(dd):
    
    name=dd[dd['developer-friendly name']=='name']["VARIABLE NAME"].astype('str').iloc[0]
    in_state=dd[dd['developer-friendly name']=='tuition.in_state']["VARIABLE NAME"].iloc[0]
    out_of_state=dd[dd['developer-friendly name']=='tuition.out_of_state']["VARIABLE NAME"].iloc[0]

    a=dd[dd['developer-friendly name']=='program.bachelors.engineering']
    b=a[a['Description'].str.contains("Program offered",regex=False)].iloc[0]
    bach_eng=b["VARIABLE NAME"]
    offered=float(b["Code"])
    #(name,in_state,out_of_state,bach_eng,offered)
    
    return (name, in_state, out_of_state, bach_eng, offered)

In [18]:
# Test Cell: `get_dd_info_test`
import hashlib

info_tuple = get_dd_info(m_df)
print(info_tuple)
assert len(info_tuple) == 5, "Your tuple is not of length 5"

m = hashlib.md5()
for t in info_tuple:
    m.update(str(t).encode('utf-8'))
hash_fn = m.hexdigest()
assert hash_fn == "29e0f9a03ae1d011723538a17c624665"
print("Passed!")

('INSTNM', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'CIP14BACHL', 1.0)
Passed!


#### Exercise 5 (2 points)

Now that you know which variable names to use in the full data file, use that to your advantage.

Write a function that, given the file name and the tuple of variable names and codes:

1. imports only the columns of interest in the `Most-Recent-Cohorts-All-Data-Elements.csv`, not all 140MB of data.
2. keeps only the school that offer a bachelor's in engineering 
3. adds a new column, use `new_col_name` inside your funtion, which is the percent difference in in-state and out-of-state tuition, using the equation below

$PercentDifference = \frac{(OutOfStateTuition - InStateTuition)}{InStateTuition}$

__Note:__ The first 4 elements of the tuple, 'names_codes' consists of the names of the columns that we wish to import. The last element of the tuple (5th element) is the Code of schools that offer a bachelor's in engineering/engineering technologies. 

The __`.assign()`__ method must be used for adding the new column

Hint: Remember to reset the index! 

In [19]:
def percent_diff(fname, names_codes, new_col_name):
    cols=list(names_codes)[0:4]
    Code=list(names_codes)[4]
    data=pd.read_csv(fname)[cols]
    data=data[data[cols[3]]==Code]
    perc_diff_df=data.assign(mydiff=lambda x: (x['TUITIONFEE_OUT']-x['TUITIONFEE_IN'])/x['TUITIONFEE_IN'])
    perc_diff_df.rename(columns={"mydiff":new_col_name},inplace=True)
    perc_diff_df.reset_index(drop=True,inplace=True)
    return perc_diff_df

In [20]:
# Test Cell: `percent_diff_test`

p_d_df = percent_diff('../resource/lib/publicdata/mt2p1/Most-Recent-Cohorts-All-Data-Elements.csv', 
                      info_tuple, 
                      'tuition.difference')

#print(hash_pandas_object(p_d_df).sum())
assert 'tuition.difference' in p_d_df.columns, "You have not added the column, 'tuition.difference' to the dataframe"
assert hash_pandas_object(p_d_df).sum() == 3804987697303416746, "You have imported the tuition data incorrectly"

print("Percent difference values seem correct. But did you use `assign()?` Let's see...")

# Tests that you actually used `apply()` in your function:

def assign_fail():
    raise ValueError("Did you really use assign?")
    
setattr(pd.DataFrame, 'assign', assign_fail)
try:
    percent_diff('../resource/lib/publicdata/mt2p1/Most-Recent-Cohorts-All-Data-Elements.csv', info_tuple, 'tuition.difference')
except (ValueError, TypeError):
    print("You used `assign()`!")
else:
    assert False, "Are you sure you used `assign()`?"
finally:
    setattr(pd.DataFrame, 'assign', SAVE_ASSIGN)
    
display(p_d_df.head())
print("\n Passed!")


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (6,9,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1008,1009,1010,1011,1014,1015,1016,1017,1018,1019,1021,1022,1023,1027,1028,1029,1030,1031,1032,1034,1035,1036,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1065,1066,1067,1068,1069,1070,1071,1073,1074,1075,1076,1078,1079,1080,1081,1082,1083,1084,1086,1087,1088,1089,1091,1092,1093,1094,1095,1096,1097,1099,1100,1101,1102,1104,1105,1106,1107,1108,1109,1110,1112,1113,1114,1115,1118,1119,1121,1122,1123,1125,1127,1128,1131,1132,1134,1135,1136,1138,1140,1141,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1170,1171,1172,1173,1174,1175,1177,1178,1179,1180,1183,1184,1185,1186,1187,1188,1190,1192,1196,1199,1200,1201,1209,1212,1213,1214,1222,1223,1224,1225,1226,1227,1235,1236,1237,123

AssertionError: You have imported the tuition data incorrectly

#### Exercise 6 ( 2 points)

For the final product, let's make this dataframe more user-friendly.

Write a function that

1. sorts descending by the tuition difference, `tuition.difference`
2. resets the index to provide a clear "ranking" without the old index being included in the dataframe
3. renames the columns using the data dictionary 'developer-friendly name' instead of the 'VARIABLE NAME'

In [ ]:
def friendlify(unsorted_df, sort_column_name, data_dictionary):
    
    friendly_df=unsorted_df.copy()
    friendly_df.sort_values(by=sort_column_name,ascending=False,inplace=True)
    friendly_df.reset_index(drop=True,inplace=True)

    clu=dict(zip(data_dictionary['VARIABLE NAME'],data_dictionary['developer-friendly name']))
    cols=list(friendly_df)
    
    for old_col in cols[0:-1]:
        new_col=clu[old_col]
        friendly_df.rename(columns={old_col:new_col},inplace=True)
    
    cols=list(friendly_df)

    return friendly_df

In [45]:
# Test Cell: `friendlify_test`

f_df = friendlify(p_d_df, 
                  'tuition.difference',
                  m_df)

for col_name in info_tuple[:-1]:
    assert col_name not in f_df.columns, "You have not renamed the columns"
assert hash_pandas_object(f_df).sum() == 3602324263510516280, "You have not manipulated the data correctly"
print("Passed!")

AssertionError: You have not manipulated the data correctly

**Fin!** You've reached the end of this problem. Don't forget to restart the kernel and run the entire notebook from top-to-bottom to make sure you did everything correctly. If that is working, try submitting this problem. (Recall that you *must* submit and pass the autograder to get credit for your work!)